<h4><b>Interactive Animation plot<b><h4>
<hr>

In [2]:
import k3d
import vtk
import glob
import numpy as np
import pyvista as pv
from vtk.util import numpy_support

<h6><b>Function for loading vtm files<b><h6>
<hr>

In [4]:
def vtm_to_Poly (name):
    """Returns the vtkPolyData object.

    Args:
        name: address of the vtm file

    Returns:
        vtkPolyData
    """    
    filename = name
    reader = vtk.vtkXMLMultiBlockDataReader()
    reader.SetFileName(filename)
    reader.Update()
    
    geometryFilter = vtk.vtkGeometryFilter()
    geometryFilter.SetInputData(reader.GetOutput().GetBlock(0).GetBlock(0))
    geometryFilter.Update()
    return geometryFilter.GetOutput()

<h6><b>Function for mesh triangulation<b><h6>
<hr>

In [ ]:
def triangulate(poly_data):
    """Create a triangulate version of the polyData.
       This is necessary for consistency of the grids at different time-steps
       since "k3d.vtk_poly_data" does this step by default.
       check: https://github.com/K3D-tools/K3D-jupyter/blob/main/k3d/factory.py
       lines: 1898-1902 
    
    Args:
        vtkPolyData 

    Returns:
        vtkPolyData (triangle)
    """
    if (max(poly_data.GetPolys().GetMaxCellSize(), poly_data.GetStrips().GetMaxCellSize()) > 3):
        cut_triangles = vtk.vtkTriangleFilter()
        cut_triangles.SetInputData(poly_data)
        cut_triangles.Update()
        poly_data = cut_triangles.GetOutput()
    return poly_data

<h6><b>Loading stl and vtm files<b><h6>
<hr>

In [5]:
# read the data

stldir = "examples/sandiaFlameD/mesh/stls"
STL = glob.glob (f'{stldir}/*.stl')

vtmdir = "examples/sandiaFlameD/mesh/vtms"
VTM = glob.glob (f'{vtmdir}/*.vtm')

grids = [pv.read(stl) for stl in STL]
contours = [triangulate(vtm_to_Poly (vtm)) for vtm in VTM]

<h6><b>Creating interactive animation<b><h6>
<hr>

In [ ]:
plot = k3d.plot()

#! select attribute (field) and range.
cl_at = ('T', 0.0, 2000)


plt_vtk = k3d.vtk_poly_data(contours[0] ,color_attribute = cl_at, color_map = k3d.basic_color_maps.Jet)
plt_mesh = k3d.vtk_poly_data(grids[0], opacity=0.05, wireframe=True,color = 0xaaaaaa)

plot += plt_mesh
plot += plt_vtk


attribute = {str(t):numpy_support.vtk_to_numpy(contours[t].GetPointData().GetArray(cl_at[0])) for t in range(len(contours))}
vertices = {str(t):numpy_support.vtk_to_numpy(grids[t].GetPoints().GetData()).astype(np.float32) for t in range(len(grids))}
indices = {str(t):numpy_support.vtk_to_numpy(grids[t].GetPolys().GetData()).reshape(-1, 4)[:, 1:4].astype(np.float32) for t in range(len(grids))}

# print (attribute) # check to see the embeded arrays used int contours color


plt_mesh.indices,plt_mesh.vertices = indices,vertices

plt_vtk.attribute = attribute 
plot.fps = 1  # This is important if not set to one then it will interpolate the mesh
# plot.time = -1
# plot.display()
plot

<h6><b>For creating .html files<b><h6>
<hr>

In [3]:
#! this is where the magic happens

f = open('./volume.html', 'w', encoding='UTF-8')
f.write(plot.get_snapshot(9, 'K3DInstance.startAutoPlay();'))
f.close()